In [1]:
!pip install wandb

   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.2 MB 1.5 MB/s eta 0:00:02
   --- ------------------------------------ 0.2/2.2 MB 2.1 MB/s eta 0:00:01
   -------- ------------------------------- 0.5/2.2 MB 3.1 MB/s eta 0:00:01
   ----------------- ---------------------- 0.9/2.2 MB 4.6 MB/s eta 0:00:01
   ---------------------- ----------------- 1.2/2.2 MB 5.6 MB/s eta 0:00:01
   --------------------------- ------------ 1.5/2.2 MB 5.1 MB/s eta 0:00:01
   -------------------------------- ------- 1.8/2.2 MB 5.7 MB/s eta 0:00:01
   --------------------------------- ------ 1.9/2.2 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 5.2 MB/s eta 0:00:00
Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
   ---------------------------------------- 0.0/266.1 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/2


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\sergi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!wandb login

wandb: Currently logged in as: 202316187 (sergom2). Use `wandb login --relogin` to force relogin


In [1]:
import wandb
import random


# Cargar el dataset desde una carpeta local
dataset_path = 'C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/training'  # Asegúrate de cambiar esto por la ruta real

# Transformaciones comunes para un dataset de imágenes
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Cargar el dataset
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)


# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="ML2-img-class",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.001,
    "architecture": "CNN",
    "dataset": dataset,
    "epochs": 20,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 202316187 (sergom2). Use `wandb login --relogin` to force relogin


acc,▃▁▄▆▇▇▇█
loss,█▄▂▁▃▁▂▂
acc,0.82706
loss,0.21024


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import wandb

# Configuración inicial de wandb
wandb.init(project="ML2-img-class", name="modelo1", config={
    "learning_rate": 0.001,
    "architecture": "resnet18",
    "dataset": "Tu Dataset Local",
    "epochs": 10,
})
config = wandb.config

# Transformaciones para el dataset
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Cargando el dataset
dataset_path = 'C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/training'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Definir el modelo
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(dataset.classes))  # Ajusta según el núm. de clases en tu dataset

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

# Entrenamiento del modelo
model.train()
for epoch in range(config.epochs):
    running_loss = 0.0
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    # Calcula la pérdida promedio y registra en wandb
    epoch_loss = running_loss / len(dataloader)
    wandb.log({"epoch": epoch, "loss": epoch_loss})

    print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}')

# Guardar el modelo entrenado
torch.save(model.state_dict(), 'model.pth')
wandb.save('model.pth')

# [Opcional] Terminar el run de wandb
wandb.finish()


c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 1.0212
Epoch 2, Loss: 0.5974


KeyboardInterrupt: 

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import wandb

# Configuración inicial de wandb
wandb.init(project="ML2-img-class", name="ModeloPrueba", config={
    "learning_rate": 0.001,
    "architecture": "resnet18",
    "dataset": "Tu Dataset Local",
    "epochs": 5,
})
config = wandb.config

# Transformaciones para el conjunto de entrenamiento y validación
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

valid_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Cargando los datasets
train_dataset_path = 'C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/training'
valid_dataset_path = 'C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/validation'

train_dataset = datasets.ImageFolder(root=train_dataset_path, transform=train_transform)
valid_dataset = datasets.ImageFolder(root=valid_dataset_path, transform=valid_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

# Definir el modelo
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))  # Ajusta según el número de clases en tu dataset

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

# Entrenamiento del modelo
model.train()
for epoch in range(config.epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    # Calcula la pérdida promedio de entrenamiento y registra en wandb
    epoch_loss = running_loss / len(train_loader)
    wandb.log({"epoch": epoch, "train_loss": epoch_loss})

    # Fase de validación
    model.eval()  # Pone el modelo en modo de evaluación
    valid_loss = 0.0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
    
    # Calcula la pérdida promedio de validación y registra en wandb
    epoch_valid_loss = valid_loss / len(valid_loader)
    wandb.log({"epoch": epoch, "valid_loss": epoch_valid_loss})

    print(f'Epoch {epoch+1}, Train Loss: {epoch_loss:.4f}, Valid Loss: {epoch_valid_loss:.4f}')

    model.train()  # Vuelve a poner el modelo en modo de entrenamiento

# Guardar el modelo entrenado
torch.save(model.state_dict(), 'modelo1.pth')
wandb.save('modelo1.pth')

# [Opcional] Terminar el run de wandb
wandb.finish()


epoch,▁▁
train_loss,▁
valid_loss,▁
epoch,0
train_loss,1.06381
valid_loss,1.02227


c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Train Loss: 1.0857, Valid Loss: 1.0428
Epoch 2, Train Loss: 0.6036, Valid Loss: 1.0348
Epoch 3, Train Loss: 0.4784, Valid Loss: 0.9035
Epoch 4, Train Loss: 0.2952, Valid Loss: 0.7149
Epoch 5, Train Loss: 0.2381, Valid Loss: 1.4741


OSError: [WinError 1314] El cliente no dispone de un privilegio requerido: 'C:\\Users\\sergi\\OneDrive\\Escritorio\\MBD\\Machine Learning 2\\Practicas_DeepLearning_2024\\streamlit\\modelo1.pth' -> 'c:\\Users\\sergi\\OneDrive\\Escritorio\\MBD\\Machine Learning 2\\Practicas_DeepLearning_2024\\streamlit\\wandb\\run-20240408_181409-syx7eidi\\files\\modelo1.pth'

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
import torch.nn as nn
import torch.optim as optim
import numpy as np
import wandb

# Configuración inicial de wandb
wandb.init(project="ML2-img-class", name="Modelo1Grande", config={
    "learning_rate": 0.0001,
    "architecture": "resnet50",
    "dataset": "Tu Dataset Local",
    "named_children": "20 capas",
    "epochs": 25,
})
config = wandb.config

# Transformaciones
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

valid_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/training', transform=train_transforms)
valid_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/validation', transform=valid_transforms)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=128, shuffle=False)

model = torchvision.models.resnet50(pretrained=True)

# Descongelar las últimas 25 capas
for name, child in list(model.named_children())[-20:]:
    for param in child.parameters():
        param.requires_grad = True

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_data.classes))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.1)

def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return correct / total

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_accuracy = 0.0
        
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_accuracy += calculate_accuracy(outputs, labels) * inputs.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = running_accuracy / len(train_loader.dataset)
        
        # Validación
        model.eval()
        val_running_loss = 0.0
        val_running_accuracy = 0.0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_running_loss += loss.item() * inputs.size(0)
                val_running_accuracy += calculate_accuracy(outputs, labels) * inputs.size(0)
                
        val_epoch_loss = val_running_loss / len(valid_loader.dataset)
        val_epoch_accuracy = val_running_accuracy / len(valid_loader.dataset)
        
        scheduler.step()
        
        # Log a W&B
        wandb.log({'train_loss': epoch_loss, 'train_accuracy': epoch_accuracy,
                   'val_loss': val_epoch_loss, 'val_accuracy': val_epoch_accuracy})
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}, Val Loss: {val_epoch_loss:.4f}, Val Accuracy: {val_epoch_accuracy:.4f}')
        
    return model

model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)
torch.save(model.state_dict(), 'model_1_grande.pth')


c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
import torch.nn as nn
import torch.optim as optim
import numpy as np
import wandb
import time  # Importar para la medición del tiempo

# Configuración inicial de wandb
wandb.init(project="ML2-img-class", name="Modelo1Grande", config={
    "learning_rate": 0.0001,
    "architecture": "resnet50",
    "dataset": "Tu Dataset Local",
    "named_children": "20 capas",
    "epochs": 25,
})
config = wandb.config

# Transformaciones
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

valid_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/training', transform=train_transforms)
valid_data = datasets.ImageFolder('C:/Users/sergi/OneDrive/Escritorio/MBD/Machine Learning 2/Practicas_DeepLearning_2024/03TransferLearning/dataset/validation', transform=valid_transforms)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=128, shuffle=False)

model = torchvision.models.resnet50(pretrained=True)

# Descongelar las últimas 20 capas
for name, child in list(model.named_children())[-20:]:
    for param in child.parameters():
        param.requires_grad = True

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_data.classes))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.1)

def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs.data, 1)
    total = labels.size(0)
    correct = (predicted == labels).sum().item()
    return correct / total

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_accuracy = 0.0
        
        start_time_epoch = time.time()  # Tiempo de inicio de la época
        
        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_accuracy += calculate_accuracy(outputs, labels) * inputs.size(0)
            
            if i % 10 == 0:  # Imprime cada 10 lotes
                print(f'Época {epoch+1}, Lote {i+1}, Pérdida del Lote: {loss.item()}')
        
        end_time_epoch = time.time()  # Tiempo al final de la época
        epoch_duration = end_time_epoch - start_time_epoch
        estimated_time_left = epoch_duration * (num_epochs - epoch - 1)
        print(f"Tiempo estimado restante: {estimated_time_left/60:.2f} minutos")
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = running_accuracy / len(train_loader.dataset)
        
        # Validación
        model.eval()
        val_running_loss = 0.0
        val_running_accuracy = 0.0
        with torch.no_grad():
            for inputs, labels in valid_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_running_loss += loss.item() * inputs.size(0)
                val_running_accuracy += calculate_accuracy(outputs, labels) * inputs.size(0)
                
        val_epoch_loss = val_running_loss / len(valid_loader.dataset)
        val_epoch_accuracy = val_running_accuracy / len(valid_loader.dataset)
        
        scheduler.step()
        
        # Log a W&B
        wandb.log({'train_loss': epoch_loss, 'train_accuracy': epoch_accuracy,
                   'val_loss': val_epoch_loss, 'val_accuracy': val_epoch_accuracy})
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}, Val Loss: {val_epoch_loss:.4f}, Val Accuracy: {val_epoch_accuracy:.4f}')
        
    return model

model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)
torch.save(model.state_dict(), 'model_1_grande.pth')


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 202316187 (sergom2). Use `wandb login --relogin` to force relogin


c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sergi\anaconda3\envs\ML2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Época 1, Lote 1, Pérdida del Lote: 2.7204110622406006
Época 1, Lote 11, Pérdida del Lote: 1.2857760190963745
Época 1, Lote 21, Pérdida del Lote: 0.7381751537322998
Tiempo estimado restante: 590.80 minutos


KeyboardInterrupt: 